In [ ]:
!sudo apt-get update
!sudo apt-get install -y openjdk-11-jdk --no-install-recommends
!java -version # Verify installation
!export JAVA_HOME=$(java -XshowSettings:properties -version 2>&1 > /dev/null | grep 'java.home' | awk '{print $3}')
!echo $JAVA_HOME
!mkdir drivers
!cd drivers
!wget https://github.com/looker-open-source/calcite-avatica/releases/download/avatica-1.26.0-looker/avatica-1.26.0-looker.jar
!pip install langchain-looker-agent langchain-openai python-dotenv

In [ ]:
import os
import logging

# It's good practice for examples to configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

from langchain_looker_agent import LookerSQLDatabase, LookerSQLToolkit, create_looker_sql_agent
from langchain_openai import ChatOpenAI # Example LLM
from langchain.memory import ConversationBufferMemory # For conversational agent
from langchain.agents import AgentExecutor 

In [ ]:
JDBC_FULL_DRIVER_PATH = "/home/jupyter/drivers/avatica-1.26.0-looker.jar"
os.environ["OPENAI_API_KEY"] = "" ## OPEN AI API KEY
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
LOOKER_INSTANCE_URL = "" ## e.g. "https://rittman.eu.looker.com"
LOOKML_MODEL_NAME = "" ## e.g. "analytics"
CLIENT_ID = "" ## API 3.0 CLIENT ID
CLIENT_SECRET = ""  ## API 3.0 CLIENT_SECRET

db = LookerSQLDatabase(
    looker_instance_url=LOOKER_INSTANCE_URL,
    lookml_model_name=LOOKML_MODEL_NAME,
    client_id=CLIENT_ID,
    client_secret=CLIENT_SECRET,
    jdbc_driver_path=JDBC_FULL_DRIVER_PATH,
    sample_rows_in_table_info=0 # Disable samples for quickstart simplicity
    )

logger.info("LookerSQLDatabase initialized.")

In [ ]:
llm = None
memory = None
looker_toolkit = None

if db and db._connection: 
    try:
        logger.info("Initializing LLM...")
        llm = ChatOpenAI(model="gpt-4o", temperature=0) 
        logger.info(f"LLM initialized: {llm.model_name}")

        logger.info("Initializing ConversationBufferMemory...")
        memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)
        logger.info("ConversationBufferMemory initialized.")

        logger.info("Initializing LookerSQLToolkit...")
        looker_toolkit = LookerSQLToolkit(db=db)
        logger.info("LookerSQLToolkit initialized.")
    except Exception as e:
        logger.error(f"Error initializing LLM, Memory, or Toolkit: {e}", exc_info=True)
else:
    logger.warning("Skipping LLM, Memory, and Toolkit initialization as database connection failed.")

agent_executor = None
if llm and looker_toolkit and memory:
    try:
        logger.info("Creating the Looker SQL Agent Executor...")
        agent_executor = create_looker_sql_agent(
            llm=llm,
            toolkit=looker_toolkit,
            verbose=True, 
            top_k=10, 
            agent_executor_kwargs={
                "memory": memory,
                "handle_parsing_errors": True, 
                "max_iterations": 10 # Increased for potentially complex reasoning
            }
        )
        logger.info("Looker SQL Agent Executor created successfully.")
    except Exception as e:
        logger.error(f"Failed to create Looker SQL Agent Executor: {e}", exc_info=True)
else:
    logger.warning("Skipping Agent Executor creation due to missing LLM, Toolkit, or Memory.")

def run_agent_test_case(question: str, agent_exec: AgentExecutor, test_name: str):
    """Helper function to run a single test case for the agent."""
    if not agent_exec:
        logger.warning(f"AGENT TEST SKIPPED ({test_name}): Agent Executor not initialized.")
        print(f"\n--- SKIPPING TEST: {test_name} (Agent not initialized) ---")
        print(f"QUESTION: {question}")
        return "Agent not initialized."
        
    logger.info(f"\n--- Running Agent Test: {test_name} ---")
    print(f"\n--- Running Agent Test: {test_name} ---")
    print(f"QUESTION: {question}")
    
    # Persona/task instruction for the agent for this specific query
    # Note: The main SQL syntax rules are already in the agent's system prompt.
    instruction_prefix = f"""You are a helpful data analyst for Rittman Analytics, querying data through Looker.
The user is asking about data available in our Looker instance via the model '{LOOKML_MODEL_NAME}'.
If the question involves a calculation on a LookML measure (often indicated by `MEASURE<TYPE>` in the schema), 
ensure you wrap that measure with `AGGREGATE()`. For example, `SELECT AGGREGATE(\`view.measure_field\`) ...`.
Question: """
    
    full_question_for_agent = instruction_prefix + question

    try:
        # AgentExecutor with memory handles chat_history implicitly if memory was passed during its creation.
        # For isolated tests, we still pass an empty chat_history to ensure the prompt is satisfied.
        response = agent_exec.invoke({
            "input": full_question_for_agent,
            "chat_history": [] # Provide for stateless test invocation
        }) 
        output = response.get("output", "No 'output' field in agent response.")
        print(f"\nAGENT RESPONSE for '{test_name}':\n{output}")
        logger.info(f"Agent response for '{test_name}': {output[:500]}{'...' if len(output) > 500 else ''}")
        return output
    except Exception as e:
        logger.error(f"ERROR invoking agent for test '{test_name}' (Question: '{question}'): {e}", exc_info=True)
        print(f"ERROR during agent invocation for '{test_name}': {e}")
        return f"Error: {e}"
    finally:
        print("-" * 70)

# %%
if agent_executor:
    logger.info("\n--- Executing Pre-determined Agent Tests ---")

    # --- Test Case 1: List available Explores ---
    run_agent_test_case(
        question="List all available tables I can query.",
        agent_exec=agent_executor,
        test_name="List Explores"
    )

    # --- Determine an Explore to use for subsequent tests ---
    explore_for_subsequent_tests = None
    if db and db._connection:
        try:
            available_explores = list(db.get_usable_table_names())
            if available_explores:
                # Prefer 'web_sessions_fact' or 'chart_of_accounts_dim' if available, else first one
                if "web_sessions_fact" in available_explores:
                    explore_for_subsequent_tests = "web_sessions_fact"
                elif "chart_of_accounts_dim" in available_explores:
                    explore_for_subsequent_tests = "chart_of_accounts_dim"
                else:
                    explore_for_subsequent_tests = available_explores[0]
                logger.info(f"Will use Explore '{explore_for_subsequent_tests}' for subsequent targeted tests.")
            else:
                logger.warning("No usable Explores found by db object; some tests might be skipped or use placeholders.")
        except Exception as e:
            logger.error(f"Could not dynamically get Explore names for tests: {e}")
    
    if not explore_for_subsequent_tests:
        logger.warning("explore_for_subsequent_tests is not set. Subsequent tests might fail if they rely on it.")
        explore_for_subsequent_tests = "your_placeholder_explore" # Fallback to a placeholder

    # --- Test Case 2: Describe a specific Explore ---
    run_agent_test_case(
        question=f"Describe the table named '{explore_for_subsequent_tests}'. What are its columns and provide a few sample rows if possible?",
        agent_exec=agent_executor,
        test_name=f"Describe Explore '{explore_for_subsequent_tests}'"
    )

    # --- Test Case 3: Simple COUNT query ---
    run_agent_test_case(
        question=f"How many records are in the '{explore_for_subsequent_tests}' table?",
        agent_exec=agent_executor,
        test_name=f"Count Records in '{explore_for_subsequent_tests}'"
    )

    # --- Advanced Tests: Adapt these field names to YOUR Looker model ---
    # Find these in the schema output from Test Case 2 or Looker UI.
    # These are examples based on your previous logs for 'web_sessions_fact'
    # If explore_for_subsequent_tests is different, these MUST be updated.
    
    measure_field_for_test = "`web_events_fact.total_page_views`" # Example, ensure this is a valid measure in explore_for_subsequent_tests
    dimension_field_for_test = "`web_sessions_fact.device_category`" # Example dimension
    dimension_value_for_test = "desktop" # Example value for the dimension

    if explore_for_subsequent_tests == "chart_of_accounts_dim": # Example of adapting if explore changes
        measure_field_for_test = "`general_ledger_fact.net_amount`" # Assuming this is a measure in chart_of_accounts_dim
        dimension_field_for_test = "`chart_of_accounts_dim.account_class`"
        dimension_value_for_test = "ASSET" # A plausible value
        logger.info(f"Adapting advanced test fields for explore: {explore_for_subsequent_tests}")


    # --- Test Case 4: Query a LookML Measure using AGGREGATE() ---
    run_agent_test_case(
        question=f"What is the total of measure {measure_field_for_test} from the {explore_for_subsequent_tests} Explore?",
        agent_exec=agent_executor,
        test_name=f"Aggregate Measure {measure_field_for_test}"
    )

    # --- Test Case 5: Measure aggregated by a dimension ---
    run_agent_test_case(
        question=f"Show me the total {measure_field_for_test} grouped by {dimension_field_for_test} from the {explore_for_subsequent_tests} Explore.",
        agent_exec=agent_executor,
        test_name=f"Aggregate Measure by Dimension"
    )
    
    # --- Test Case 6: Measure with a WHERE clause on a dimension ---
    run_agent_test_case(
        question=f"What is the total {measure_field_for_test} for {dimension_field_for_test} = '{dimension_value_for_test}' from the {explore_for_subsequent_tests} Explore?",
        agent_exec=agent_executor,
        test_name=f"Aggregate Measure with Filter"
    )

    # --- Test Case 7: Query that should NOT use JOIN, subquery, or window function ---
    run_agent_test_case(
        question=f"List the top 3 {dimension_field_for_test} by total {measure_field_for_test} from {explore_for_subsequent_tests}, ordered descending by that total.",
        agent_exec=agent_executor,
        test_name="Top N without Window Functions"
    )

else:
    logger.error("Agent Executor not initialized. Cannot run pre-determined agent tests.")
    print("ERROR: Agent Executor could not be initialized. Please review setup cells for errors.")


# ## Phase 7: Cleanup (Optional) 

# %%
if db:
    try:
        logger.info("Attempting to close Looker database connection...")
        db.close()
    except Exception as e:
        logger.error(f"Error closing Looker connection: {e}", exc_info=True)
else:
    logger.info("No database connection (db object) to close or it was not initialized.")

logger.info("--- End of Looker SQL Agent Comprehensive Tests Notebook ---")